In [3]:
import os

import sys
import argparse
import random
import math
import re
import time
import numpy as np
import cv2
import json
import matplotlib
import matplotlib.pyplot as plt
import skimage.transform
import time
os.environ['CUDA_VISIBLE_DEVICES']="0,1,2,3"




def main():
    ROOT_DIR = os.path.abspath("../")

    # Import Mask
    sys.path.append(ROOT_DIR)
    from mrcnn.config import Config
    from mrcnn import utils
    import mrcnn.model as modellib
    from mrcnn import visualize
    from mrcnn.model import log
    from satellite import SatelliteDataset, SatelliteConfig

 
    # config dataset
    config = SatelliteConfig()
    # config.NUM_CLASSES = 31
    # re-initialize the config class
    # config.__init__()
    config.display()


    dataset_train = SatelliteDataset(dup_flag=False)
    dataset_train.config_dataset_with_big_map(root_dir="/home/ubuntu/mask_rcnn/data/big_image"
                                            )
 

    print(len(dataset_train.image_info))
    # prepare dataset
    dataset_train.prepare()

    dataset_val = SatelliteDataset(dup_flag=False)
    dataset_val.config_dataset_by_super_category(dataset_name=dataset_name, 
                             json_file=train_json,
                             skip_classes=['Road','Body_Of_water','Tree'],
                             root_dir=image_root_dir,
                             instance_number_threshold=instance_number_threshold,
                             area_set=['001', '002', '003',\
                                       '004', '005', '006',\
                                       '007']
                             )

    dataset_val.prepare()
    print(len(dataset_val.image_info))
    print(len(dataset_val.image_info))

    assert True, "stop"

    # create model
    model = modellib.MaskRCNN(mode='training', config=config, model_dir=MODEL_DIR)
    init_with = 'coco'

    if init_with == 'imagenet':
        model.load_weights(model.get_imagenet_weights(), by_name=True)
    elif init_with == 'coco':
        model.load_weights(COCO_MODEL_PATH, by_name=True,
                       exclude=['mrcnn_class_logits', 'mrcnn_bbox_fc', 'mrcnn_bbox', "mrcnn_mask"])
    elif init_with == 'last':
        model.load_weights(model.find_last(), by_name=True)

    # Skip training, Only do validation.
    if mode == 'train': 
        model.train(dataset_train, dataset_val,
            learning_rate=config.LEARNING_RATE,
            epochs=5,
            layers='heads')
        # fine-tune
        model.train(dataset_train, dataset_val,
            learning_rate=config.LEARNING_RATE /10,
            epochs=10,
            layers='all')
        print("Finish training")

    print("Validation")
    class InferenceConfig(SatelliteConfig):
        GPU_COUNT = 1
        IMAGES_PER_GPU = 1
    inference_config = InferenceConfig()
    # inference_config.GPU_COUNT = 1
    # inference_config.IMAGES_PER_GPU = 1
    # inference_config.BATCH_SIZE = 1
    model = modellib.MaskRCNN(mode='inference',
                          config=inference_config,
                          model_dir=MODEL_DIR)

    model_path = model.find_last()
    print("Load weghts from ", model_path)
    model.load_weights(model_path, by_name=True)



# Evaluation

    APs = []
    # for image_id in dataset_val.image_ids:
    for image_id in dataset_train.image_ids:
        begin_time = time.time()
        
        start_time = time.time()
        image, image_meta, gt_class_id, gt_bbox, gt_mask = \
            modellib.load_image_gt(dataset_train, inference_config,
                                   image_id, use_mini_mask=False)
        print("Data loading time: {}".format(time.time() - start_time))
        molded_images = np.expand_dims(modellib.mold_image(image, inference_config), 0)
        # print(model.config.BATCH_SIZE)
        start_time = time.time()
        resuts = model.detect([image], verbose=0)
        print("Inference time: {}".format(time.time() - start_time))
        r = resuts[0]
        start_time = time.time()
        AP, precisions, recalls, overlaps = \
            utils.compute_ap(gt_bbox, gt_class_id, gt_mask,
                    r['rois'], r["class_ids"], r["scores"], r["masks"])
        print("Evaluation time: {}".format(time.time() - start_time))
        APs.append(AP)
        print("overall time: {}".format(time.time() - begin_time))

    print("mAP: ", np.mean(APs))

if __name__ == '__main__':
    main()


Configurations:
BACKBONE                       resnet101
BACKBONE_STRIDES               [4, 8, 16, 32, 64]
BATCH_SIZE                     16
BBOX_STD_DEV                   [0.1 0.1 0.2 0.2]
COMPUTE_BACKBONE_SHAPE         None
DETECTION_MAX_INSTANCES        100
DETECTION_MIN_CONFIDENCE       0.7
DETECTION_NMS_THRESHOLD        0.3
FPN_CLASSIF_FC_LAYERS_SIZE     1024
GPU_COUNT                      4
GRADIENT_CLIP_NORM             5.0
IMAGES_PER_GPU                 4
IMAGE_CHANNEL_COUNT            3
IMAGE_MAX_DIM                  1024
IMAGE_META_SIZE                31
IMAGE_MIN_DIM                  1024
IMAGE_MIN_SCALE                0
IMAGE_RESIZE_MODE              square
IMAGE_SHAPE                    [1024 1024    3]
LEARNING_MOMENTUM              0.9
LEARNING_RATE                  0.001
LOSS_WEIGHTS                   {'rpn_class_loss': 1.0, 'rpn_bbox_loss': 1.0, 'mrcnn_class_loss': 1.0, 'mrcnn_bbox_loss': 1.0, 'mrcnn_mask_loss': 1.0}
MASK_POOL_SIZE                 14
MASK_SHAPE       

NameError: name 'dup_flag' is not defined